<a href="https://colab.research.google.com/github/Vaycold/Python_ML/blob/main/Clustering/E-commerce%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E-Commerce 
_____________

## Load the data

In [1]:
# Install the package
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings(action = 'ignore')
pd.set_option('display.max_columns', None)

In [4]:
# data url
user_meta_url = 'https://raw.githubusercontent.com/dajeong-lecture/raw_data/main/user_meta_data.csv'
user_statsdata_url = 'https://raw.githubusercontent.com/dajeong-lecture/raw_data/main/user_stats_data.csv'

user_meta_df = pd.read_csv(user_meta_url, parse_dates=[0])
user_stats_df = pd.read_csv(user_statsdata_url, parse_dates=[0])

In [11]:
user_meta_df.columns

Index(['user_id', 'sex', 'birthday_year', 'membership_type', 'category_prefer',
       'joined_date', 'deleted_date', 'join_path', 'os', 'recommended_cnt'],
      dtype='object')

     Meta Data
      - user_id         : 고객 고유 ID
      - sex             : 성별(남0, 여1)
      - birthday_year   : 생일년도(yyyy)
      - membership_type : 멤버쉽타입(100 : 비회원, 300. 500: 정회원)
      - category_prefer : 카테고리 선호 입력수
      - joined_date     : 가입일자(yyyy)
      - deleted_date    : 
      - join_path       :
      - os              :
      - recommended_cnt :

In [8]:
# Data shape
print(user_meta_df.shape)
print(user_stats_df.shape)

(3412, 10)
(3412, 11)


In [9]:
user_meta_df.head(2)

,user_id,sex,birthday_year,membership_type,category_prefer,joined_date,deleted_date,join_path,os,recommended_cnt
0,KjIRvUKVTgxGaek,0,1995,300,3,2019-09-09,NaT,NaN,AOS,4
1,QWhJIG1fOkhUJzG,1,1995,300,2,2020-04-13,NaT,NaN,AOS,1


In [10]:
user_stats_df.head(2)

,user_id,membership_type,total_order_cnt,total_cost,avg_coupon_amt,avg_point_amt,weekly_order_cnt,monthly_order_cnt,avg_cost,avg_item_cost,last_date
0,q3qlSo1pvyacJoy,100,1.0,23800.0,0.0,0.0,1.0,1.0,23800.0,0.0,2020-04-05
1,JRy7QRK0UMFP81X,100,1.0,23000.0,0.0,0.0,1.0,1.0,23000.0,10000.0,2020-01-17
